# Headers

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import itertools as itert
from pyquil import Program
import pyquil.api as api
from pyquil.quil import DefGate
from pyquil.gates import *
from pyquil import get_qc
from pyquil.quilatom import unpack_qubit

# Input possibilities

In [2]:
def get_possibilities (n):
    # whole- entire set of possible n input values. Totally there are 2**n values.
    whole=[]
    # this will ensure that all binary values obtained have same length
    max_binary_length= len(bin(2**n-1)[2:])
    # creating binary values 
    for i in range(0, 2**n):
        # this will represent actual binary value to be obtained
        curr_binary=[0]*max_binary_length
        # This gives current length of binary. Why is this required?
        # SO bin(0) return 0b0 and bin(8) returns 0b1000-> different lengths. 
        # But we want all our binary values to have same length for simplicity 
        corresponding_binary= list(bin(i))[2:] # bin returns string, we are turning it into character array/list
        # [2:] disregards the initial 0b in 0bx (returned by bin) and considers x which is the binary value  
        curr_length= len(corresponding_binary)
        for j in range(0, curr_length):
            corresponding_binary[j]=int(corresponding_binary[j]) # converting character array to int array
        # final binary value obtained
        curr_binary[max_binary_length-curr_length:]= corresponding_binary
        # adding this value as a possibility
        whole.append(curr_binary)
    print("Input Possibilities: ", whole)
    return whole

# -Z_0 for G

In [3]:
def createz0 (n):
    matrix_z= np.eye(2**(n))
    matrix_z[0][0]=-1
    #for i in range (0,2**n):
        #matrix_z[i][i] *= (-1)
    print (" Minus Matrix Z0:")
    print(np.negative(matrix_z))
    return np.negative(matrix_z)

# Create Z_f

In [4]:
class Uf:  
    # multiple x st f(x)=1
    def __function_generator(self,n):
        # a- binary string of length n
        #any one bit as 1  out of n bit
        curr_index= np.random.randint(low=0,high=n,size=1)[0]
        a=[0]*n
        a[curr_index]=1
        print ("This is the (randomly chosen) value of a: ", a)
        return a
    # function with only one string st f(x)=1
    def __unique_generator(self,n):
        # a- binary string of length n
        curr_a= np.random.randint(low=0,high=2,size=n)
        print ("This is the (randomly chosen) value of a: ", curr_a)
        return curr_a

    #this function creates the blackbox oracle Uf matrix for a given function f that is parameterised for input size n
    def __blackboxUf(self,n,a,fn):
        uf_matrix = np.eye(2**n)
        all_binary= get_possibilities (n)
        
        #Function_generator
        if fn==0:
            #unique generator
            for i in all_binary:
                curr_ind= all_binary.index(i)
                if (np.array_equal(a,i)):
                    #|x> to -|x> when f(x)=1
                    uf_matrix[curr_ind][curr_ind] *= (-1)  
        else:
            for i in range(0,2**n):
                curr_dot_product=0
                for k in range (0,n):
                    curr_dot_product += a[k] * int(all_binary[i][k])
                curr_dot_product %= 2
                #|x> to -|x> when f(x)=1
                if curr_dot_product == 1 :
                    uf_matrix[i][i] *= (-1)
        
        print ("UF\n", uf_matrix)
        return uf_matrix
    def createUf(self,n, fn):
        #curr_a= self.__function_generator(n)
        if (fn==0):
            print ("Only one x st f(x)=1")
            curr_a= self.__unique_generator(n)
        else:
            print ("One or more x st f(x)=1")
            curr_a= self.__function_generator(n)
        return self.__blackboxUf(n,curr_a,fn)

# Main Circuit

In [5]:
def runMainCircuit():
   
    # Taking the input n from the user
    n= int(input("Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: "))
    
    # one extra bit as our helper bit 
    #n = n+1
    
    #Number of Iterations
    num_of_iterations = max(1, int(round((np.pi/4)*np.sqrt(2**n)-1/2)))
    
    print("Number of iterations: ", num_of_iterations)

    # creating an instance of Uf
    uf = Uf()
    
    # defining start of PyQuil program
    p = Program()
    
    qc_name = "{}q-qvm".format(n)
    
    # Get our QuantumComputer instance, with a Quantum Virutal Machine (QVM) backend
    
    qc = get_qc(qc_name)
    qc.compiler.client.timeout = 600 # number of seconds/ Increasing time out 
    
    # Time taken by program needs to be checked.
    start = time. time()
    
    # creating our Uf matrix 
    # Selecting fn mapping: unique x or multiple x st f(x)=1
    fn_type=np.random.randint(low=0,high=2,size=1)[0]
    #creating uf or zf
    UfMatrix = uf.createUf(n,fn_type)
    
    GateName = "UF_GATE_GROVER"
    
    #Zero Matrix
    matrix_z0= createz0(n)
    Gate0="minus_Z0"
    
    #defining a gate using its name and matrix
    
    uf_gate_definition = DefGate(GateName, UfMatrix)
    qubits = [unpack_qubit(i) for i in range(0,n)]
    
    z0_gate = DefGate(Gate0,matrix_z0)
    
    # adding Hadamard gates to all qubits
    for i in range(0,n):
        p += H(i)
    # adding Uf gate
    for k in range(0,num_of_iterations):
        print("Iteration no: ", k)
        p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits))
        for i in range(0,n):
                p += H(i)
        p+=Program(z0_gate,Gate(name=Gate0, params=[],qubits=qubits))
        for i in range(0,n):
                p += H(i)
        print("Program uptil now: ", p) 
      
    
    
    # measurement result    
    results = qc.run_and_measure(p, trials=5)
    
    print("Results: ")
    print(results)
    
    end = time. time()
    print("Time taken by program: ", end-start)

# Testing

## n=1 

In [12]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 1
Number of iterations:  1
One or more x st f(x)=1
This is the (randomly chosen) value of a:  [1]
Input Possibilities:  [[0], [1]]
UF
 [[ 1.  0.]
 [ 0. -1.]]
 Minus Matrix Z0:
[[ 1. -0.]
 [-0. -1.]]
Iteration no:  0
Program uptil now:  DEFGATE UF_GATE_GROVER:
    1.0, 0
    0, -1.0

DEFGATE minus_Z0:
    1.0, 0
    0, -1.0

H 0
UF_GATE_GROVER 0
H 0
minus_Z0 0
H 0

Results: 
{0: array([1, 0, 1, 1, 0])}
Time taken by program:  0.14661026000976562


## n=2

In [11]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 2
Number of iterations:  1
One or more x st f(x)=1
This is the (randomly chosen) value of a:  [0, 1]
Input Possibilities:  [[0, 0], [0, 1], [1, 0], [1, 1]]
UF
 [[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]
 Minus Matrix Z0:
[[ 1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]
Iteration no:  0
Program uptil now:  DEFGATE UF_GATE_GROVER:
    1.0, 0, 0, 0
    0, -1.0, 0, 0
    0, 0, 1.0, 0
    0, 0, 0, -1.0

DEFGATE minus_Z0:
    1.0, 0, 0, 0
    0, -1.0, 0, 0
    0, 0, -1.0, 0
    0, 0, 0, -1.0

H 0
H 1
UF_GATE_GROVER 0 1
H 0
H 1
minus_Z0 0 1
H 0
H 1

Results: 
{0: array([0, 0, 0, 0, 0]), 1: array([1, 1, 1, 1, 0])}
Time taken by program:  0.3420863151550293


In [8]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 2
Number of iterations:  1
Only one x st f(x)=1
This is the (randomly chosen) value of a:  [0 0]
Input Possibilities:  [[0, 0], [0, 1], [1, 0], [1, 1]]
UF
 [[-1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
 Minus Matrix Z0:
[[ 1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]
Iteration no:  0
Program uptil now:  DEFGATE UF_GATE_GROVER:
    -1.0, 0, 0, 0
    0, 1.0, 0, 0
    0, 0, 1.0, 0
    0, 0, 0, 1.0

DEFGATE minus_Z0:
    1.0, 0, 0, 0
    0, -1.0, 0, 0
    0, 0, -1.0, 0
    0, 0, 0, -1.0

H 0
H 1
UF_GATE_GROVER 0 1
H 0
H 1
minus_Z0 0 1
H 0
H 1

Results: 
{0: array([0, 0, 0, 0, 0]), 1: array([0, 0, 0, 0, 0])}
Time taken by program:  0.34009218215942383
